In [ ]:
import os
import random
import base64
import numpy as np
from dotenv import load_dotenv
load_dotenv()
# import streamlit as st
from datetime import datetime
from pathlib import Path
from pydantic.v1 import BaseModel, Field

from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import AgentType, initialize_agent, create_structured_chat_agent
from langchain.agents import Tool
from langchain.globals import set_debug
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain, RetrievalQA
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_openai import OpenAIEmbeddings
from langchain_groq import ChatGroq
from groq import Groq
from openai import OpenAI

# from byaldi import RAGMultiModalModel
from langchain_core.messages import SystemMessage, HumanMessage

In [3]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
def image_chat(image_path:str, query:str, model_name:str):
    base64_image = encode_image(image_path)
    client = Groq()
    # client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

    chat_completion  = client.chat.completions.create(
        model=model_name,
        messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": f"{query}"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                    },
                },
            ],
        }
    ],
        temperature=0,
        max_tokens=1024,
        top_p=1,
        stream=False,
        stop=None,
    )
    return chat_completion.choices[0].message.content

In [24]:
image_path = "./data/invoice_01.png"
model_name = "llama-3.2-11b-vision-preview"
query = """첨부 파일의 주요 내용을 key, value pair json 형식으로 요약해주세요.
"""

result = image_chat(image_path=image_path, query=query, model_name=model_name)
print(result)

Sure, here is a key-value pair representation of the invoice:

{
    "ISSUED TO": "Richard Sanchez",
    "ISSUED TO ADDRESS": "Thynk Unlimited 123 Anywhere St., Any City",
    "INVOICE NO": "01234",
    "DATE": "11.02.2030",
    "DUE DATE": "11.03.2030",
    "PAY TO": "Barclays Bank",
    "PAY TO ACCOUNT NAME": "Adeline Palmerston",
    "PAY TO ACCOUNT NO": "0123 4567 8901",
    "DESCRIPTION": {
        "Brand consultation": 100,
        "logo design": 100,
        "Website design": 100,
        "Social media templates": 100,
        "Brand photography": 100,
        "Brand guide": 100
    },
    "UNIT PRICE": 100,
    "QTY": 1,
    "TOTAL": 100,
    "SUBTOTAL": 400,
    "TAX": 10,
    "TOTAL": 440
}
